# 深度学习框架 PyTorch Lightning 和医学图像分析框架 MONAI 教程

## Lightning AI

<img alt="Lightning" src="images/PyTorch-to-Fabric-Spectrum.png" width="2500">

### <img alt="Lightning Fabric" src="images/Lightning-Fabric.png" width="500">

Lightning Fabric 是一种快速且轻量的方法，用于扩展 PyTorch 模型，无需编写大量代码。您可以在 5 行代码内将 PyTorch 代码转换为 Lightning Fabric，从而获得对最先进的分布式训练功能（如 DDP、FSDP、DeepSpeed、混合精度等）的访问，以扩展最大的十亿参数模型。

Fabric 与 Lightning 的完全不同的几个关键方面包括：快速实现（无需重构代码），最大化灵活性（编写自己的训练和/或推理逻辑），以及最大化控制（一切都是可选的，可根据需要逐步添加更多功能）。

### <img alt="PyTorch Lightning" src="images/Pytorch-Lightning.png" width="500">

PyTorch Lightning 是一个轻量级的基于 PyTorch 的高层次模型接口，它提供了一种简单的方式来组织 PyTorch 代码，使得代码更加模块化、可读性更强、可维护性更高。它的设计目标是让研究人员专注于模型的设计，而不是训练过程的实现。

它的核心思想是将训练过程分为 5 个部分：`LightningModule`、`LightningDataModule`、`Trainer`、`Callbacks` 和 `LightningLogger`。

- `LightningModule` 是模型的核心，它包含了模型的定义、前向传播、损失函数、优化器等；
- `LightningDataModule` 是数据的核心，它包含了数据的加载、预处理、划分等；
- `Trainer` 是训练过程的核心，它包含了训练过程的超参数、优化器、学习率调整策略、训练过程的配置等；
- `Callbacks` 是训练过程的钩子函数，它包含了训练过程中的一些回调函数，如模型保存、学习率调整、训练过程可视化等；
- `LightningLogger` 是训练过程的日志记录器，它包含了训练过程中的一些日志记录，如训练过程的可视化、训练过程的日志记录等。

最新的 PyTorch Lightning 教程包括如何在 Cifar10 上训练 Resnet 以达到 94% 的准确率，如何使用 DataModules，以及如何使用 Fine-Tuning Scheduler 来微调模型。


## <img alt="MONAI" src="images/MONAI.png" width="500">

MONAI (Medical Open Network for AI) 是一个开源的医学图像分析框架，它旨在提供一个用于医学图像分析的端到端的深度学习框架，以加速 AI 在医学图像分析领域的研究和应用。

MONAI 提供了一系列的工具，如数据加载、数据预处理、数据划分、模型定义、模型训练、模型评估、模型推理等，它们都是基于 PyTorch 实现的，因此可以很方便地与 PyTorch 结合使用。MONAI 教程可以在其 GitHub 页面上找到，提供了丰富的资源和示例，以帮助用户快速上手和应用 MONAI。


### Lightning 的安装

Lightning 的安装非常简单。

若想用 pip 安装，可以使用以下命令：

```bash
pip install lightning
```

若想用 conda 安装，可以使用以下命令：

```bash
conda install lightning -c conda-forge
```

安装完成后，可以使用以下命令查看版本信息：

```bash
lightning --version
```

In [1]:
!python -c "import pytorch_lightning,lightning_fabric" || pip install lightning

### PyTorch 代码实现 MNIST 分类

下面我们以训练一个简单的 MNIST 分类模型为例，来介绍如何从 PyTorch 迁移到 Fabric 和 Lightning。

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from tqdm import tqdm

# Set the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else
                      "mps" if torch.backends.mps.is_available() else
                      "cpu")


# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the input tensor
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# Define the training parameters
batch_size = 64
learning_rate = 0.01
num_epochs = 10
validation_split = 0.2

# Load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
dataset = datasets.MNIST(root=os.getcwd(), train=True, transform=transform, download=True)

# Split the dataset into training and validation sets
train_size = int((1 - validation_split) * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

test_dataset = datasets.MNIST(root=os.getcwd(), train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model and move it to the device
model = Net().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    with tqdm(total=len(train_loader), desc=f"Epoch {epoch + 1}/{num_epochs}", unit="batch") as pbar:
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)

            # Update the progress bar
            pbar.update(1)
            pbar.set_postfix({"Train Loss": train_loss / ((pbar.n - 1) * train_loader.batch_size + images.size(0))})

    # Validation
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for val_images, val_labels in val_loader:
            val_images, val_labels = val_images.to(device), val_labels.to(device)

            val_outputs = model(val_images)
            val_loss += criterion(val_outputs, val_labels).item()

            _, val_predicted = torch.max(val_outputs.data, 1)
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    val_loss /= len(val_loader)

    # Print validation metrics
    tqdm.write(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

# Test the model
model.eval()
total_correct = 0
total_samples = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

# Print the accuracy on the test set
accuracy = 100 * total_correct / total_samples
print(f"Test Accuracy: {accuracy:.2f}%")

# Save the model checkpoint
torch.save(model.state_dict(), "mnist.ckpt")

# Load the model checkpoint
model.load_state_dict(torch.load("mnist.ckpt"))


Epoch 1/10: 100%|██████████| 750/750 [00:05<00:00, 149.22batch/s, Train Loss=1.16]


Validation Loss: 0.4706, Validation Accuracy: 87.14%


Epoch 2/10: 100%|██████████| 750/750 [00:04<00:00, 158.99batch/s, Train Loss=0.417]


Validation Loss: 0.3566, Validation Accuracy: 90.03%


Epoch 3/10: 100%|██████████| 750/750 [00:04<00:00, 157.96batch/s, Train Loss=0.349]


...

Epoch 9/10: 100%|██████████| 750/750 [00:04<00:00, 159.83batch/s, Train Loss=0.221]


Validation Loss: 0.2196, Validation Accuracy: 93.62%


Epoch 10/10: 100%|██████████| 750/750 [00:04<00:00, 160.39batch/s, Train Loss=0.207]


Validation Loss: 0.2129, Validation Accuracy: 93.74%
Test Accuracy: 94.13%


<All keys matched successfully>

这份 PyTorch 代码实现了一个用于手写数字识别的神经网络模型。代码主要包括以下几个部分：

1. **导入所需的库**：包括 os、torch 以及 torch 的相关模块、数据集模块以及进度条模块。
2. **设置设备**：根据可用的硬件情况，将模型的计算设备设置为 CPU 或 GPU。
3. **定义神经网络模型的架构**：定义了一个包含三个全连接层的模型，输入大小为 784，输出大小为 10。
4. **定义训练参数**：包括批处理大小、学习率、训练周期数和验证集的比例。
5. **加载 MNIST 数据集**：使用 torchvision 中的 datasets 模块加载 MNIST 数据集，并进行数据预处理。
6. **将数据集划分为训练集和验证集**：使用 random_split 函数将数据集划分为训练集和验证集。
7. **创建数据加载器**：使用 DataLoader 模块创建训练集、验证集和测试集的数据加载器，用于批量加载数据。
8. **初始化模型并将其移动到设备上**：创建模型实例，并将模型移动到之前设置的设备上。
9. **定义损失函数和优化器**：使用交叉熵损失函数和随机梯度下降（SGD）优化器。
10. **训练循环**：对于每个训练周期，迭代训练集中的批次数据进行训练。计算输出、损失、梯度，然后更新模型参数。
11. **验证过程**：在每个训练周期后，使用验证集评估模型的性能。计算验证集上的损失和准确率。
12. **测试模型**：使用测试集评估模型的性能。计算测试集上的准确率。
13. **保存和加载模型**：保存训练好的模型权重参数到文件中，并可以加载之前保存的模型。

总体来说，这份代码实现了一个基于 PyTorch 的手写数字识别模型的训练、验证和测试过程，并提供了保存和加载模型的功能。

从上面的 PyTorch 代码可以看出，即使是一个简单的 MNIST 分类模型，也涉及了多个步骤，从数据处理到模型训练。尽管 PyTorch 为我们提供了强大的灵活性，但在一些场景下，代码的简洁性和扩展性可能受限。这正是 Lightning Fabric 发挥作用的地方。Fabric 作为一个新的、开源的库，使得在保持对训练循环完全控制的同时，快速且轻松地扩展模型成为可能。

Fabric 在以下几个方面优于纯 PyTorch 代码：

* **最小化更改需求**：Fabric 被引入到 PyTorch Lightning 中，旨在以最少的代码更改加速 PyTorch 的训练或推理代码，这使得它非常适合在现有的 PyTorch 项目中使用，以加速和扩展模型，而不需要大规模的重构。
* **快速实施**：与 Lightning 的完整训练器相比，Fabric 实现起来更快。不需要重新构建代码，只需在 PyTorch 脚本中更改几行代码，就可以利用 Fabric 的功能。
* **灵活性和控制**：Fabric 允许进行灵活的迭代式训练、元学习、交叉验证和其他类型的优化算法，而不需要深入了解框架内部。
* **自动化设备和精度支持**：Fabric 自动将模型和数据放置到设备上，并支持混合和双精度，从而减小内存占用。

下面我们来看看如何用 Fabric 改进这个模型。

[When to Use PyTorch Lightning or Lightning Fabric](https://lightning.ai/blog/pytorch-lightning-and-fabric)

尽管 Fabric 提供了一些显著的优势，比如简化代码和扩展模型的能力，但在可读性和维护性方面可能仍然存在一些挑战。为了进一步提高代码的可读性、可维护性和模块化，最终迁移到 PyTorch Lightning 就显得尤为重要。PyTorch Lightning 提供了一个更高层次的抽象，使得研究人员可以专注于模型的设计，而不是训练过程的实现，从而在代码的组织和管理上实现质的飞跃。

下面我们来看看如何用 Lightning 重构这个模型。

### LightningDataModule

首先，我们需要将数据的加载、预处理、划分等操作封装到 `LightningDataModule` 中，这样可以使得数据的加载、预处理、划分等操作更加模块化、可读性更强、可维护性更高。

In [3]:
import os
import torch
import torchvision
from torchvision import datasets, transforms
import lightning.pytorch as pl
from torch.utils.data import DataLoader, random_split


class MNISTDataModule(pl.LightningDataModule):
    def __init__(self, data_dir: str = os.getcwd(), batch_size: int = 64, num_workers: int = 4):
        super().__init__()
        self.data_dir = data_dir
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])
        self.batch_size = batch_size
        self.num_workers = num_workers

    def prepare_data(self):
        # Download the MNIST dataset
        datasets.MNIST(self.data_dir, train=True, download=True)
        datasets.MNIST(self.data_dir, train=False, download=True)

    def setup(self, stage=None):
        # Assign train/val/test datasets for use in dataloaders
        if stage == 'fit' or stage is None:
            mnist_full = datasets.MNIST(self.data_dir, train=True, transform=self.transform)
            self.mnist_train, self.mnist_val = random_split(mnist_full, [55000, 5000])
        if stage == 'test' or stage is None:
            self.mnist_test = datasets.MNIST(self.data_dir, train=False, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=self.batch_size, num_workers=self.num_workers,
                          persistent_workers=True, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=self.batch_size, num_workers=self.num_workers,
                          persistent_workers=True, shuffle=False)

    def test_dataloader(self):
        return DataLoader(self.mnist_test, batch_size=self.batch_size, num_workers=self.num_workers,
                          persistent_workers=True, shuffle=False)

这样一来，我们就可以将数据的加载、预处理、划分等操作从模型中分离出来，使得模型的定义更加简洁。

### LightningModule

接下来，我们需要将模型的定义、前向传播、损失函数、优化器等操作封装到 `LightningModule` 中，这样可以使得模型的定义、前向传播、损失函数、优化器等操作更加模块化、可读性更强、可维护性更高。

In [4]:
import lightning.pytorch as pl
import torch.nn as nn
import torch


# Define the neural network architecture
class MNISTModule(pl.LightningModule):
    def __init__(self, learning_rate=0.01):
        super().__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.loss = nn.CrossEntropyLoss()
        self.lr = learning_rate

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the input tensor
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def configure_optimizers(self):
        # Define the optimizer
        optimizer = torch.optim.SGD(self.parameters(), lr=self.lr)
        return optimizer

    def training_step(self, batch, batch_idx):
        # extract x and y from the batch
        x, y = batch

        # forward pass
        logits = self(x)

        # calculate the loss
        loss = self.loss(logits, y)

        # Logging to the built-in logger
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)

        return loss

    def validation_step(self, batch, batch_idx):
        # extract x and y from the batch
        x, y = batch

        # forward pass
        logits = self(x)

        # calculate the loss
        loss = self.loss(logits, y)

        # calculate the accuracy
        _, predicted = torch.max(logits.data, 1)
        correct = (predicted == y).sum().item()

        # Logging to the built-in logger
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', correct / len(y), prog_bar=True)

        return loss

    def test_step(self, batch, batch_idx):
        # extract x and y from the batch
        x, y = batch

        # forward pass
        logits = self(x)

        # calculate the loss
        loss = self.loss(logits, y)

        # calculate the accuracy
        _, predicted = torch.max(logits.data, 1)
        correct = (predicted == y).sum().item()

        # Logging to the built-in logger
        self.log('test_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('test_acc', correct / len(y), on_step=True, on_epoch=True, prog_bar=True, logger=True)

        return {'test_loss': loss, 'correct': correct, 'total': len(y)}


### Trainer

在定义好 `LightningDataModule` 和 `LightningModule` 后，我们就可以使用 `Trainer` 来训练模型了。

`Trainer` 是 Lightning 提供的一个用于训练模型的类，它封装了训练过程中的大量细节，使得训练过程更加简单。

In [5]:
# Initialize data module and model
data_module = MNISTDataModule()
model = MNISTModule()

callbacks = [
    # Early stopping callback to prevent overfitting
    pl.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min'),
    # Model checkpoint callback to save the best model(s)
    pl.callbacks.ModelCheckpoint(monitor='val_loss')
]

# Initialize trainer
trainer = pl.Trainer(
    max_epochs=20,  # Number of epochs, if -1, runs indefinitely
    precision='16-mixed',  # precision of training, default is 32-bit
    callbacks=callbacks,  # callbacks defined above
    accelerator='cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
    # device to use for training
)

# Tune the model hyperparameters
from lightning.pytorch.tuner import Tuner

tuner = Tuner(trainer)

# Find the optimal batch size that maximizes GPU utilization
tuner.scale_batch_size(model, data_module)

# Find the "optimal" learning rate (BE CAREFUL)
tuner.lr_find(model, data_module)

# Fit the model
trainer.fit(model, data_module)

Using 16bit Automatic Mixed Precision (AMP)
/Users/rizhong/miniconda3/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:124: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/rizhong/miniconda3/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
/Users/rizhong/miniconda3/lib/python3.11/site-packages/torch/amp/auto

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=100` reached.
Learning rate set to 0.47863009232263803
Restoring states from the checkpoint path at /Users/rizhong/Documents/r/100400_数据挖掘/.lr_find_21918648-cefb-4b94-ae7b-6412fbc08198.ckpt
Restored all states from the checkpoint at /Users/rizhong/Documents/r/100400_数据挖掘/.lr_find_21918648-cefb-4b94-ae7b-6412fbc08198.ckpt

  | Name | Type             | Params
------------------------------------------
0 | fc1  | Linear           | 100 K 
1 | fc2  | Linear           | 8.3 K 
2 | fc3  | Linear           | 650   
3 | loss | CrossEntropyLoss | 0     
------------------------------------------
109 K     Trainable params
0         Non-trainable params
109 K     Total params
0.438     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [6]:
# Test the model
trainer.test(model, data_module)

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_acc_epoch       │     0.97079998254776      │
│      test_loss_epoch      │    0.14197485148906708    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss_epoch': 0.14197485148906708, 'test_acc_epoch': 0.97079998254776}]

这种方式使用 PyTorch Lightning 框架可以大幅简化代码，提高模型的可读性和可维护性，同时保持了灵活性和强大的功能。

### MONAI 的安装

MONAI 的安装非常简单。

若想用 pip 安装，可以使用以下命令：

```bash
pip install monai
```

若想用 conda 安装，可以使用以下命令：

```bash
conda install monai -c conda-forge
```

安装完成后，可以使用以下命令查看版本信息：

```bash
python -c "import monai; print(monai.__version__)"
```

In [7]:
!python -c "import monai" || pip install monai-weekly